In [25]:
from tkinter import * 
from tkinter import filedialog,messagebox
import cv2
import os
import numpy as np
import pandas as pd
import csv
import scipy.stats
import math

In [2]:
from keras import applications
from keras.models import Model
from keras.models import load_model
from keras.layers import Input,concatenate

C:\Users\hp\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
img_width, img_height, channels = 224, 224, 3
dim = (img_width, img_height)

In [4]:
model = applications.vgg19.VGG19(weights='imagenet', include_top=False, pooling='max')
for layer in model.layers[:13]:
    layer.trainable = False

In [5]:
anchor_in = Input(shape=(img_width, img_height, channels))
pos_in = Input(shape=(img_width, img_height, channels))
neg_in = Input(shape=(img_width, img_height, channels))

anchor_out = model(anchor_in)
pos_out = model(pos_in)
neg_out = model(neg_in)
merged_vector = concatenate([anchor_out, pos_out, neg_out],axis=1)

model = Model(inputs=[anchor_in, pos_in, neg_in], outputs=merged_vector)

In [6]:
model.load_weights('C:/Users/hp/Python notebooks/Hacks/Axis bank/models/model_weights_56_59.h5')

In [7]:
def to_rgb(img):
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
    img_rgb = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
    return img_rgb

def returnimages(path):
    image=cv2.imread(path)                  #bringing the image
    image=cv2.resize(image, (img_width, img_height))
    image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image=to_rgb(image).reshape(1,img_width, img_height,3)/255.0       #resizing and normalizing    
    return image



In [22]:
def openaccount():
    
    global dirc
    global e
    
    
    top.withdraw()
    acc = Tk(className='Accepting a new Signature')
    

    
    #getting the image
    messagebox.showinfo("Title", "Please select the signature of the account holder")
    img = filedialog.askopenfilename()
    anchor = returnimages(img)
    
    
    #passing image through model    
    x=model.predict([anchor,anchor,anchor])
    x=x[0,0:512]
    
    
    #getting account number!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    accno=e.get()
    
    
    #setting directory to bankdetails folder and creating one if it doesn't exist in the specified mod
    y=dirc+accno+".csv"
    columns="512 magic"
    np.savetxt(y,x, header=columns,comments="")
    
    #destroying acc activity
    acc.destroy()
    
    #showing back the basic top activity
    top.deiconify()

In [23]:
def checkaccount():
    top.withdraw()
    
    global dirc
    global e
#    string = dirc.get()
    
    ck= Tk(className='Checking a Signature against an account no')
    
    #getting the directory of bank details file - to be done only once, figure out how
    #messagebox.showinfo("Title", "Please select the directory pointing to bankdetails folder")
    #dirc = filedialog.askdirectory()
    
    #getting the image
    messagebox.showinfo("Check", "Please select the image to check")
    check_img = filedialog.askopenfilename()
    check = returnimages(check_img)
    
    #passing image through model
    x=model.predict([check,check,check])
    x=x[0,0:512]
    
    #getting account number!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    accno=e.get()
    
    a=pd.read_csv(dirc+"/"+accno+".csv")
    t=np.array([])
    for i in range(512):
        t=np.append(t,a.loc[i])
    dist=np.linalg.norm(t-x)
    
    mu = 38
    variance = 15.99
    sigma = math.sqrt(variance)
    
    if(dist>=38):
        t=1-scipy.stats.norm.pdf(dist, mu, sigma)*10
        print("Prob of it being fake is ",60+(40*t))
    else:
        t=1-scipy.stats.norm.pdf(dist, mu, sigma)*10
        print("Prob of it being genuine is ",60+(40*t))
    
    
    
    
    
    #destroying check activity
    ck.destroy()
    
    #showing back the basic top activity
    top.deiconify()

In [26]:
dirc=""
#mod="D:/Hacks/Axis bank signatures/"
    
def closetop():
    top.destroy()
    
top = Tk(className='Axis Signature Verification Dashboard')
# Code to add widgets will go here...
#top.withdraw() # Close the root window

a = Button(top, text = 'Open a New Account', width = 25 , height = 5,command=openaccount)
b = Button(top, text = 'Check a signature', width = 25 , height = 5,command=checkaccount)
c = Button(top, text = 'Close', width = 25 , height = 5,command=closetop)

messagebox.showinfo("Title", "Please select the place where you'd like the bankdetails to be stored or where the bankdetails folder exists")
dirc = filedialog.askdirectory()

if dirc.endswith("bankdetails"):
    dirc=dirc+"/"
elif dirc.endswith("bankdetails/"):
    dirc=dirc
else:
    os.makedirs(dirc+"bankdetails")
    dirc=dirc+"bankdetails/"
    
w = Label(top, text="Please enter the account number you wish to work with in the text area")
w.pack()

e = Entry(top)
e.pack()
e.focus_set()
    




a.pack()
b.pack()
c.pack()

mainloop()

Prob of it being fake is  83.91575735574193
43.39076613593331
Prob of it being genuine is  60.71858471800182
37.289330083219404
Prob of it being genuine is  99.5074580427086
26.144910723124248


KeyboardInterrupt: 

In [ ]:
3